In [82]:
import pandas as pd
import numpy as np

data = pd.read_csv('LMM_data1.csv')

times = data['t'].values
fwd0 = data['fwd'].values

#print(fwd0)
days = 252

periods = 5
Nums = periods 
sim = 5
fwd = np.zeros((sim, Nums, periods))
Dev = np.zeros((sim, Nums))
Timestep = np.zeros(periods + 1)
correl = np.zeros(periods + 1)
fwdvol = np.zeros(periods + 1)
Notional = np.zeros(periods + 1)
fixed_rate = np.zeros(periods + 1)
df = np.zeros((sim, Nums, periods + 1))
MtM = np.zeros((sim, periods))



for k in range(periods + 1):
    Timestep[k] = 1
    correl[k] = 1
    fwdvol[k] = 0.2
    fixed_rate[k]= 0.05
    Notional[k] = 1000000
    
#print(Timestep)


for i in range(sim):
    Orgsumfxd = 0
    Orgsumflt = 0
    df[i, 0, 0] = 1
    j = 0
    for k in range(periods):
        fwd[i, 0, k] = fwd0[k]
        df[i, j, k + 1] = np.exp(-fwd[i, j, k] * Timestep[k]) * df[i, j, k]
        Orgsumfxd += Notional[k] * fixed_rate[k] * df[i, j, k + 1]
        Orgsumflt += Notional[k] * fwd[i, j, k] * df[i, j, k + 1]

    #print(Orgsumfxd)
    #print(Orgsumflt)
    
    MtM[i, 0] = Orgsumfxd - Orgsumflt

    for j in range(1, periods):
        drift = 0   
        sumfxd = 0
        sumflt = 0
        for k in range(j, periods):
            Z1 = np.random.normal(0,1)
            drift += (Timestep[k-1] * fwd[i, j-1, k] * fwdvol[j] * correl[k-1]) / (1 + fwd[i, j-1, k] * Timestep[k-1]) 
            fwd[i, j, k] = fwd[i, j-1, k] * np.exp((drift * fwdvol[k-1] - 0.5 * fwdvol[k-1] ** 2) * Timestep[k-1] + fwdvol[k-1] * Timestep[k-1] ** 0.5 * Z1)
            df[i, j, j] = 1
            df[i, j, k + 1] = np.exp(-fwd[i, j, k] * Timestep[k]) * df[i, j, k]
            
            sumfxd += Timestep[k] * Notional[k] * fixed_rate[k] * df[i, j, k + 1]
            sumflt += Timestep[k] * Notional[k] * fwd[i, j, k] * df[i, j, k + 1]
            
        MtM[i, j] = sumfxd - sumflt
    
print(df)
print(fwd)

print(MtM)


EPE = np.mean(np.where(MtM > 0, MtM, 0), axis=0)
#print(EPE)

import pandas as pd

df_df = pd.DataFrame(df.reshape(sim, -1))  
fwd_df = pd.DataFrame(fwd.reshape(sim, -1))  

with pd.ExcelWriter('output.xlsx') as writer:
    df_df.to_excel(writer, sheet_name='DF', index=False)
    fwd_df.to_excel(writer, sheet_name='FWD', index=False)


[[[1.         0.99004983 0.96078944 0.91393119 0.85214379 0.77880078]
  [0.         1.         0.97501741 0.92352943 0.85566181 0.78021545]
  [0.         0.         1.         0.95194143 0.8783523  0.7821567 ]
  [0.         0.         0.         1.         0.93108135 0.82314131]
  [0.         0.         0.         0.         1.         0.91728642]]

 [[1.         0.99004983 0.96078944 0.91393119 0.85214379 0.77880078]
  [0.         1.         0.97069324 0.92251628 0.87204352 0.79279335]
  [0.         0.         1.         0.95669784 0.9012684  0.80426085]
  [0.         0.         0.         1.         0.94124933 0.85186118]
  [0.         0.         0.         0.         1.         0.91048678]]

 [[1.         0.99004983 0.96078944 0.91393119 0.85214379 0.77880078]
  [0.         1.         0.97089094 0.92163452 0.85414141 0.76188137]
  [0.         0.         1.         0.9544709  0.88458095 0.78350255]
  [0.         0.         0.         1.         0.89733646 0.74093342]
  [0.         0.

PermissionError: [Errno 13] Permission denied: 'output.xlsx'

In [12]:
import os
current_path = os.getcwd()
print(current_path)

C:\Users\Laptop\pricing_models
